# Pré processamento do dataset de tarefas

In [10]:
import pandas as pd
import numpy as np

In [11]:
try:
    df = pd.read_csv(
        '../data/tasks.csv',
        encoding='utf-8',
        sep=",",
        quotechar='"',
        on_bad_lines='skip',      # ignora linhas corrompidas
        dtype=str,               # carrega tudo como string, depois ajusta
        engine='c'          # usa engine python que é mais tolerante
    )
    print(f"CSV carregado com engine C! {len(df)} linhas encontradas.")

except Exception as e:
    print(f"Erro ao ler CSV: {e}")

CSV carregado com engine C! 7232 linhas encontradas.


In [12]:
# Verificar informações básicas do dataset
if df is not None:
    print("=== INFORMAÇÕES BÁSICAS DO DATASET ===")
    print(f"Shape: {df.shape}")
    print(f"\nTipos de dados:")
    print(df.dtypes)
    print(f"\nPrimeiras 5 linhas:")
    print(df.head())
    print(f"\nValores nulos por coluna:")
    print(df.isnull().sum())
else:
    print("DataFrame não foi carregado corretamente.")

=== INFORMAÇÕES BÁSICAS DO DATASET ===
Shape: (7232, 17)

Tipos de dados:
SUMMARY                          object
DTSTART                          object
DTEND                            object
DUE                              object
NOTES                            object
ATTENDEE                         object
LOCATION                         object
PRIORITY                         object
URL                              object
CALENDAR                         object
UID                              object
ORGANIZER                        object
CATEGORIES                       object
DURATION                         object
REPLACES RECURRENT EVENT FROM    object
CANCELLED                        object
CREATED                          object
dtype: object

Primeiras 5 linhas:
  SUMMARY           DTSTART             DTEND  DUE NOTES ATTENDEE LOCATION  \
0       🍳  07/01/2025 08:00  07/01/2025 08:20  NaN   NaN      NaN      NaN   
1       🧋  07/01/2025 10:00  07/01/2025 10:05  NaN   Na

In [13]:
# Analisar as colunas de data especificamente
if df is not None:
    print("=== ANÁLISE DAS COLUNAS DE DATA ===")

    # Verificar formatos de data
    date_columns = ['DTSTART', 'DTEND', 'DUE', 'CREATED']

    for col in date_columns:
        if col in df.columns:
            print(f"\n{col}:")
            print(f"  - Valores únicos (sample): {df[col].dropna().unique()[:5]}")
            print(f"  - Valores nulos: {df[col].isnull().sum()}")
            print(f"  - Total não-nulos: {df[col].notna().sum()}")

    # Verificar colunas categóricas importantes
    print(f"\n=== COLUNAS CATEGÓRICAS ===")
    categorical_cols = ['CALENDAR', 'CATEGORIES', 'PRIORITY']

    for col in categorical_cols:
        if col in df.columns:
            print(f"\n{col}:")
            values = df[col].value_counts().head(10)
            print(values)

=== ANÁLISE DAS COLUNAS DE DATA ===

DTSTART:
  - Valores únicos (sample): ['07/01/2025 08:00' '07/01/2025 10:00' '07/01/2025 12:30'
 '07/01/2025 16:00' '07/01/2025 18:00']
  - Valores nulos: 0
  - Total não-nulos: 7232

DTEND:
  - Valores únicos (sample): ['07/01/2025 08:20' '07/01/2025 10:05' '07/01/2025 13:00'
 '07/01/2025 16:15' '07/01/2025 18:05']
  - Valores nulos: 2
  - Total não-nulos: 7230

DUE:
  - Valores únicos (sample): []
  - Valores nulos: 7232
  - Total não-nulos: 0

CREATED:
  - Valores únicos (sample): ['07/01/2025 10:12' '07/01/2025 10:14' '07/01/2025 10:15'
 '07/01/2025 10:16' '07/01/2025 10:17']
  - Valores nulos: 0
  - Total não-nulos: 7232

=== COLUNAS CATEGÓRICAS ===

CALENDAR:
CALENDAR
Alimentação    2323
Trabalho       1255
PUC            1219
Exercícios      870
Pessoal         564
PUC 👻           514
Saúde           171
Exames          117
Social           52
Compromisso      49
Name: count, dtype: int64

CATEGORIES:
Series([], Name: count, dtype: int64)

PR

In [14]:
# Converter colunas de data para datetime
if df is not None:
    print("=== CONVERTENDO COLUNAS DE DATA ===")

    # Função para converter data de forma segura
    def safe_datetime_convert(series, format_str='%d/%m/%Y %H:%M'):
        try:
            return pd.to_datetime(series, format=format_str, errors='coerce')
        except:
            return pd.to_datetime(series, errors='coerce')

    # Converter colunas de data
    date_columns = ['DTSTART', 'DTEND', 'CREATED']

    for col in date_columns:
        if col in df.columns:
            print(f"Convertendo {col}...")
            df[col] = safe_datetime_convert(df[col])
            print(f"  - Convertidos: {df[col].notna().sum()}")
            print(f"  - Erros/Nulos: {df[col].isna().sum()}")

    # Converter DURATION para numérico
    if 'DURATION' in df.columns:
        print("Convertendo DURATION...")
        df['DURATION'] = pd.to_numeric(df['DURATION'], errors='coerce')
        print(f"  - Convertidos: {df['DURATION'].notna().sum()}")
        print(f"  - Estatísticas: min={df['DURATION'].min():.2f}, max={df['DURATION'].max():.2f}, média={df['DURATION'].mean():.2f}")

    print("\n✅ Conversões concluídas!")

=== CONVERTENDO COLUNAS DE DATA ===
Convertendo DTSTART...
  - Convertidos: 7232
  - Erros/Nulos: 0
Convertendo DTEND...
  - Convertidos: 7230
  - Erros/Nulos: 2
Convertendo CREATED...
  - Convertidos: 7232
  - Erros/Nulos: 0
Convertendo DURATION...
  - Convertidos: 7230
  - Estatísticas: min=0.08, max=435.00, média=3.20

✅ Conversões concluídas!


### COLUNA: SUMMARY

In [ ]:
print("=== ANÁlISE DA COLUNA SUMMARY ===")

print(f"Quantidade de valores únicos: {df['SUMMARY'].nunique()}")
print(f"Valores únicos: {df['SUMMARY'].unique()}")

# Exportar valores únicos da coluna SUMMARY para uma tabela CSV e usar uma LLM para clusterizar informacoes parecidas
unique_summary = pd.DataFrame({'TITLE': df['SUMMARY'].unique()})
unique_summary.to_csv('../data/title_unique.csv', index=False, encoding='utf-8')
print("Valores únicos exportados para ../data/title_unique.csv")


=== ANÁlISE DA COLUNA SUMMARY ===
Quantidade de valores únicos: 376
Valores únicos: ['🍳' '🧋' '🍽️' '🥪' '🍌' 'INF1410 - Gerenc Proj Inf'
 'INF1608 - Análise Numérica' 'ADM1019 - Intr. Finanças'
 'INF1629 - P. Eng. de Software' 'Parabéns!' 'Alessandra' 'RM' 'P3'
 'Psicóloga' 'Zion' 'ZION' 'Autoescola' 'Aula de Violão' 'Cortar cabelo'
 'Aula de Música' 'Centro' 'Conversa PUC' 'Auto Escola: Prova Prática'
 'Detran' '🎤 Aula de Canto' '🎸 Aula de Guitarra' 'Graduação Muay Thai'
 'Visita Rede' 'Mago da Música link' 'Conversa Pedro' 'T2 Álgebra'
 'Prova Auto Escola' 'INF1301 - Relatório Plano de Ação' 'P1 INF1018'
 'P1 ProgMod' 'Entrega Trabalho INF1018' 'T3 Álgebra' 'T4 Álgebra'
 'P2 INF1018' 'P2 Álgebra' 'Trab INF1018' 'P3 Álgebra' 'P2 ProgMod'
 'Avaliação 1 Álgebra' 'P1 Cálculo' 'T1 Maple Álgebra'
 'Avaliação 2 Álgebra' 'P1 Álgebra' 'Teste 1 ProbComp' 'Av. Lab1 Cálculo'
 'Trabalho 2 IAC' 'Trabalho 3 IAC' 'Trab G1 ProbComp' 'Teste 2 ProbComp'
 'Avaliação 3 Álgebra' 'P1 IAC' 'Avaliação 4 Álgebra

In [16]:
# Salvar dataset processado
if df is not None:
    print("=== SALVANDO DATASET PROCESSADO ===")

    # Criar algumas colunas derivadas úteis
    df['YEAR'] = df['DTSTART'].dt.year
    df['MONTH'] = df['DTSTART'].dt.month
    df['DAY_OF_WEEK'] = df['DTSTART'].dt.dayofweek  # 0=Monday, 6=Sunday
    df['HOUR'] = df['DTSTART'].dt.hour
    df['DATE'] = df['DTSTART'].dt.date

    # Criar coluna de duração em minutos (se não existe DURATION)
    if 'DTSTART' in df.columns and 'DTEND' in df.columns:
        df['DURATION_MINUTES'] = (df['DTEND'] - df['DTSTART']).dt.total_seconds() / 60

    print(f"Colunas adicionadas: YEAR, MONTH, DAY_OF_WEEK, HOUR, DATE, DURATION_MINUTES")

    # Salvar como CSV processado
    output_path = '../data/tasks_processed.csv'
    df.to_csv(output_path, index=False, encoding='utf-8')
    print(f"✅ Dataset processado salvo em: {output_path}")

    # Estatísticas finais
    print(f"\n=== ESTATÍSTICAS FINAIS ===")
    print(f"Total de registros: {len(df)}")
    print(f"Período dos dados: {df['DTSTART'].min()} até {df['DTSTART'].max()}")
    print(f"Categorias únicas: {df['CALENDAR'].nunique()}")
    print(f"Duração média das tarefas: {df['DURATION_MINUTES'].mean():.1f} minutos")
else:
    print("❌ DataFrame não disponível para salvar.")

=== SALVANDO DATASET PROCESSADO ===
Colunas adicionadas: YEAR, MONTH, DAY_OF_WEEK, HOUR, DATE, DURATION_MINUTES
✅ Dataset processado salvo em: ../data/tasks_processed.csv

=== ESTATÍSTICAS FINAIS ===
Total de registros: 7232
Período dos dados: 2002-04-22 00:00:00 até 2026-12-30 15:00:00
Categorias únicas: 14
Duração média das tarefas: 192.2 minutos
